<a href="https://colab.research.google.com/github/Nix07/Utilizing-BERT-for-Aspect-Based-Sentiment-Analysis/blob/master/Data_Pre_processing_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
import json
from tqdm import tqdm
import pandas as pd

# Preparing Datasets for BERT-single

## Preparing Training Sets

In [ ]:
with open('/content/drive/My Drive/SentiHood/Original Data/sentihood-train.json', 'r') as fp:
    training_set = json.load(fp)

In [ ]:
len(training_set)

2977

In [ ]:
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

for location in locations:
    for aspect in aspects:
        df = pd.DataFrame({'id': [], 'text': [], 'sentiment': []})
        ii = 0
        for each_example in training_set:
            id = str(int(each_example['id']))
            text = each_example['text'].strip()
            
            # Add a row only of the "location" is present
            if location in text:
                aspect_found = False
                for opinion in each_example['opinions']:
                    if opinion['target_entity'] == location and opinion['aspect'] == aspect:
                        df.loc[ii] = [id, text, opinion['sentiment']]
                        aspect_found = True
                        ii += 1
                        break
                
                if not aspect_found:
                    df.loc[ii] = [id, text, 'None']
                    ii += 1

        df.to_csv('/content/drive/My Drive/SentiHood/Bert-single/TrainingData/' + str(location) + str(aspect) + '.csv', index=False)
        print(f"{location}{aspect} DONE!\tLength = {ii}")

LOCATION1dining DONE!	Length = 2977
LOCATION1general DONE!	Length = 2977
LOCATION1green-nature DONE!	Length = 2977
LOCATION1live DONE!	Length = 2977
LOCATION1multicultural DONE!	Length = 2977
LOCATION1nightlife DONE!	Length = 2977
LOCATION1price DONE!	Length = 2977
LOCATION1quiet DONE!	Length = 2977
LOCATION1safety DONE!	Length = 2977
LOCATION1shopping DONE!	Length = 2977
LOCATION1touristy DONE!	Length = 2977
LOCATION1transit-location DONE!	Length = 2977
LOCATION2dining DONE!	Length = 775
LOCATION2general DONE!	Length = 775
LOCATION2green-nature DONE!	Length = 775
LOCATION2live DONE!	Length = 775
LOCATION2multicultural DONE!	Length = 775
LOCATION2nightlife DONE!	Length = 775
LOCATION2price DONE!	Length = 775
LOCATION2quiet DONE!	Length = 775
LOCATION2safety DONE!	Length = 775
LOCATION2shopping DONE!	Length = 775
LOCATION2touristy DONE!	Length = 775
LOCATION2transit-location DONE!	Length = 775


## Preparing Validation Sets

In [ ]:
with open('/content/drive/My Drive/SentiHood/Original Data/sentihood-dev.json', 'r') as fp:
    validation_set = json.load(fp)

In [ ]:
len(validation_set)

747

In [ ]:
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

for location in locations:
    for aspect in aspects:
        df = pd.DataFrame({'id': [], 'text': [], 'sentiment': []})
        ii = 0
        for each_example in validation_set:
            id = str(int(each_example['id']))
            text = each_example['text'].strip()
            
            # Add a row only of the "location" is present
            if location in text:
                aspect_found = False
                for opinion in each_example['opinions']:
                    if opinion['target_entity'] == location and opinion['aspect'] == aspect:
                        df.loc[ii] = [id, text, opinion['sentiment']]
                        aspect_found = True
                        ii += 1
                        break
                
                if not aspect_found:
                    df.loc[ii] = [id, text, 'None']
                    ii += 1

        df.to_csv('/content/drive/My Drive/SentiHood/Bert-single/ValidationData/' + str(location) + str(aspect) + '.csv', index=False)
        print(f"{location}{aspect} DONE!\tLength = {ii}")

LOCATION1dining DONE!	Length = 747
LOCATION1general DONE!	Length = 747
LOCATION1green-nature DONE!	Length = 747
LOCATION1live DONE!	Length = 747
LOCATION1multicultural DONE!	Length = 747
LOCATION1nightlife DONE!	Length = 747
LOCATION1price DONE!	Length = 747
LOCATION1quiet DONE!	Length = 747
LOCATION1safety DONE!	Length = 747
LOCATION1shopping DONE!	Length = 747
LOCATION1touristy DONE!	Length = 747
LOCATION1transit-location DONE!	Length = 747
LOCATION2dining DONE!	Length = 190
LOCATION2general DONE!	Length = 190
LOCATION2green-nature DONE!	Length = 190
LOCATION2live DONE!	Length = 190
LOCATION2multicultural DONE!	Length = 190
LOCATION2nightlife DONE!	Length = 190
LOCATION2price DONE!	Length = 190
LOCATION2quiet DONE!	Length = 190
LOCATION2safety DONE!	Length = 190
LOCATION2shopping DONE!	Length = 190
LOCATION2touristy DONE!	Length = 190
LOCATION2transit-location DONE!	Length = 190


# Preparing Datasets for BERT-pair

**Creating Datasets for BERT-pair QA-M**

In [4]:
with open('/content/drive/My Drive/SentiHood/Original Data/sentihood-train.json', 'r') as fp:
    training_set = json.load(fp)

In [10]:
locations = ['LOCATION1', 'LOCATION2']
aspects = ['dining', 'general', 'green-nature', 'live', 'multicultural', 'nightlife', 'price', 'quiet', 'safety', 'shopping', 'touristy', 'transit-location']

df = pd.DataFrame({'id': [], 'text': [], 'auxiliary_sentence': [], 'sentiment': []})
ii = 0

for location in locations:
  for each_example in tqdm(training_set, nols=80):
    id = str(int(each_example['id']))
    text = each_example['text'].strip()

    # Add a row only of the "location" is present
    if location in text:
      current_opinion_aspects = {}
      for opinion in each_example['opinions']:
        if opinion['target_entity'] == location:
          current_opinion_aspects[opinion['aspect']] = opinion['sentiment']
      
      for aspect in aspects:
        auxiliary_sentence = f'what do you think of the {aspect} of {location}?' 
        if aspect in current_opinion_aspects.keys():
          df.loc[ii] = [id, text, auxiliary_sentence, current_opinion_aspects[aspect]]
        else:
          df.loc[ii] = [id, text, auxiliary_sentence, 'None']
        ii += 1
          

100%|██████████| 2977/2977 [00:48<00:00, 61.58it/s]


In [16]:
# df.to_csv('/content/drive/My Drive/SentiHood/Bert-pair/QA-M/training_set.csv', index=False)